<a href="https://colab.research.google.com/github/Twoy519/zed-dot-run-analysis/blob/master/Zed_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import numpy as np
import time

import gspread
from oauth2client.client import GoogleCredentials as GC
from google.colab import auth
from gspread_dataframe import set_with_dataframe

In [ ]:
auth.authenticate_user()
gc = gspread.authorize(GC.get_application_default())

In [ ]:
def get_horse(horse_number):
    try:
      r = requests.get(f'https://api.zed.run/api/v1/horses/get/{horse_number}').json()
      return r
    except:
      print(f"error on horse number {horse_number}")

In [ ]:
def scrape_horses(start_horse_number, stop_horse_number):
    return_data = []
    for i in range(start_horse_number, stop_horse_number + 1):
        horse = get_horse(i)
        horse['id'] = i
        return_data.append(horse)
        time.sleep(1)
    return return_data

In [ ]:
def get_horses(start, stop):
  horses = scrape_horses(start,stop)
  df = pd.json_normalize(horses)
  return df

In [ ]:
def sanitize_fields(horses, imporant_fields):
  for x in important_fields:
    if x not in horses:
      horses[x] = np.nan
  
  return horses

In [ ]:
def clean_horses(horses, important_fields):
  df = sanitize_fields(horses, important_fields)
  return df[important_fields]

In [ ]:
def save_horses(start, stop, clean_horses):
  title = f'horses_{start}_{stop}'
  gc.create(title)
  sheet = gc.open(title).sheet1
  set_with_dataframe(sheet, clean_horses, include_column_header=True) 
  horses.to_csv(f'{title}.csv', index=False)


In [ ]:
  # identify the fields we care about
important_fields = [
    "id",
    "bloodline",
    "breed_type",
    "genotype",
    "hash_info.name",
    "win_rate",
    "number_of_races",
    "parents.father.bloodline",
    "parents.father.breed_type",
    "parents.father.genotype",
    "parents.father.hash_info.name",
    "parents.father.win_rate",
    "parents.father.number_of_races",
    "parents.mother.bloodline",
    "parents.mother.breed_type",
    "parents.mother.genotype",
    "parents.mother.hash_info.name",
    "parents.mother.win_rate",
    "parents.mother.number_of_races"
]

In [ ]:
start = 0
stop = 999

In [ ]:
horses = get_horses(start, stop)
horses_cleaned = clean_horses(horses, important_fields)
save_horses(start, stop, horses_cleaned)

In [ ]:
save_horses(start, stop, horses_cleaned)

In [ ]:
horses_cleaned

,id,bloodline,breed_type,genotype,hash_info.name,win_rate,number_of_races,parents.father.bloodline,parents.father.breed_type,parents.father.genotype,parents.father.hash_info.name,parents.father.win_rate,parents.father.number_of_races,parents.mother.bloodline,parents.mother.breed_type,parents.mother.genotype,parents.mother.hash_info.name,parents.mother.win_rate,parents.mother.number_of_races
0,0,Nakamoto,genesis,Z1,Thin Frivolity,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Nakamoto,genesis,Z1,Chase Jackson,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Nakamoto,genesis,Z1,Wall Street,23.33,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Nakamoto,genesis,Z1,Millions,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Nakamoto,genesis,Z1,Guns and Horses,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,Nakamoto,genesis,Z2,Second Best,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,996,Nakamoto,genesis,Z2,For The Fans,0.00,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,997,Nakamoto,genesis,Z2,Come At Me,15.38,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,998,Nakamoto,genesis,Z2,My Sore Leg,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
